# Pre-requisites:
* Connection information to our team PostgreSQL database at AWS. You will need it for your config.py file that resides in the same folder as this notebook. Example:

username = "######"

password = "######"

host = "database-bootcamp.cdykxayfkga1.us-west-1.rds.amazonaws.com"

port = 5432

database = "greenVehicles"


# 1. (EXTRACT) VEHICLES REGISTERED IN CALIFORNIA BY ZIP, FUEL, MAKE, YEAR

Dataset location: https://data.ca.gov/dataset/vehicle-fuel-type-count-by-zip-code



In [1]:
# Import required modules and variables
import pandas as pd
import requests
import pprint
from config import username, password, host, port, database

In [2]:
# Reference the dataset downloaded from
# https://data.ca.gov/dataset/vehicle-fuel-type-count-by-zip-code for year 2020 vehicle data
#  in California
ca_vehicle_csv = '../template/static/data/vehicle-count-as-of-1-1-2020.csv'
green_vehicles_df = pd.read_csv(ca_vehicle_csv)
green_vehicles_df.head()

/Users/henrytirado/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Date,Zip Code,Model Year,Fuel,Make,Duty,Vehicles
0,1/1/2020,90001,2007,Gasoline,ACURA,Light,15
1,1/1/2020,90002,2007,Gasoline,ACURA,Light,20
2,1/1/2020,90003,2007,Gasoline,ACURA,Light,29
3,1/1/2020,90004,2007,Gasoline,ACURA,Light,19
4,1/1/2020,90006,2007,Gasoline,ACURA,Light,15


In [3]:
green_vehicles_df.dtypes

Date          object
Zip Code      object
Model Year    object
Fuel          object
Make          object
Duty          object
Vehicles       int64
dtype: object

In [4]:
len(green_vehicles_df)

602394

# (TRANSFORMATION): update Date to Postgres style

In [13]:
from datetime import datetime

# # Transform jobTimestamp into a date/time timestamp
# finance_trends_df['jobTimestamp'] = pd.to_datetime(finance_trends_df['jobTimestamp'], unit='ms')

# Transform startInterval into a date/time format
green_vehicles_df['Date'] = pd.to_datetime(green_vehicles_df['Date'], format='%m/%d/%Y')

In [24]:
green_vehicles_df.dtypes

Date          datetime64[ns]
Zip Code              object
Model Year            object
Fuel                  object
Make                  object
Duty                  object
Vehicles               int64
dtype: object

In [35]:
green_vehicles_df.tail()

,Date,Zip Code,Model Year,Fuel,Make,Duty,Vehicles
602389,2020-01-01,OOS,Unk,Gasoline,Unk,Heavy,98
602390,2020-01-01,90012,Unk,Gasoline,Unk,Light,12
602391,2020-01-01,93065,Unk,Gasoline,Unk,Light,11
602392,2020-01-01,95762,Unk,Gasoline,Unk,Light,12
602393,2020-01-01,OOS,Unk,Gasoline,Unk,Light,52


# 5. (LOAD) into database
Use the values pulled in from config.py to construct the database connection string.  Connect to the database.  Write the data frames to the database tables

In [22]:
from sqlalchemy import create_engine

# Define the engine string to be used for connecting to your database using the 
#  environment variables found in config.py
connection_string = f'{username}:{password}@{host}:{port}/{database}'
engine = create_engine(f'postgresql://{connection_string}')

In [29]:
# Write green_vehicles to database
green_vehicles_df.to_sql(name='vehicleCount2020', con=engine, if_exists='append', index=False)

In [30]:
# Verify that there is data in finance_trends
pd.read_sql_query("select * from vehicleCount2020", con=engine).head()

,date,zip_code,model_year,fuel,make,duty,vehicles
0,2020-01-01,90027,2018,Hybrid,Nissan,light,35
